In [ ]:
import pandas as pd
import time
from typing import cast
station_info = pd.read_csv('station_data.csv')

# divide the stations into surplus stations and deficit stations
surplus_stations = station_info[station_info['CurrentInventory'] > station_info['Optimal Inventory']]
deficit_stations = station_info[station_info['CurrentInventory'] < station_info['Optimal Inventory']]
delta_udf_marginal = dict()
# test the time for traverse every possible pair of (surplus, deficit)
start_time = time.time()
for surplus_id, surplus in surplus_stations.iterrows():
    surplus_id = cast(int, surplus_id)
    sur_current = surplus['CurrentInventory']
    sur_optimal = surplus['Optimal Inventory']
    sur_deviation = sur_current - sur_optimal
    sur_current_udf = surplus[f"UDF({sur_current})"]
    for deficit_id, deficit in deficit_stations.iterrows():
        deficit_id  = cast(int, deficit_id)
        def_current = deficit['CurrentInventory']
        def_optimal = deficit['Optimal Inventory']
        def_deviation = def_current - def_optimal
        def_current_udf = deficit[f"UDF({def_current})"]
        cap = min(sur_deviation, -def_deviation)
        for m in range(1, cap + 1):
            delta_udf = sur_current_udf - surplus[f"UDF({sur_current - m})"] + def_current_udf  - deficit[f"UDF({def_current + m})"]
            delta_udf_marginal[(surplus_id + 1, deficit_id + 1, m)] = delta_udf
end_time = time.time()
print(f"Time taken to traverse all pairs: {end_time - start_time} seconds")

# save the results to a CSV file, set the dict keys (3-element tuple) as three columns 
delta_udf_marginal_df = pd.DataFrame.from_dict(delta_udf_marginal, orient='index', columns=['Delta UDF'])
# If the index is a string like "(1,2,3)", first convert it to a tuple of integers
if isinstance(delta_udf_marginal_df.index[0], str):
    delta_udf_marginal_df.index = delta_udf_marginal_df.index.map(lambda x: tuple(map(int, x.strip("()").split(","))))
# Reset index to convert the tuple index to a column
delta_udf_marginal_df = delta_udf_marginal_df.reset_index()
# Expand the tuple in the 'index' column into three separate columns
delta_udf_marginal_df[['from_surplus', 'to_deficit', 'transfers']] = pd.DataFrame(delta_udf_marginal_df['index'].tolist(), index=delta_udf_marginal_df.index)
# Drop the original 'index' column and reorder columns
delta_udf_marginal_df = delta_udf_marginal_df[['from_surplus', 'to_deficit', 'transfers', 'Delta UDF']]
delta_udf_marginal_df.to_csv('delta_udf_marginal.csv', index=False)

Time taken to traverse all pairs: 27.03710675239563 seconds


In [11]:
# test the time of loading the dict from CSV
start_time = time.time()
# load the csv to dict with the values of from_surplus,to_deficit,transfers columns being the keys as a tuple
delta_udf_marginal_df = pd.read_csv('delta_udf_marginal.csv')
delta_udf_marginal_dict = delta_udf_marginal_df.set_index(['from_surplus', 'to_deficit', 'transfers'])['Delta UDF'].to_dict()
end_time = time.time()
print(f"Time taken to load the CSV file: {end_time - start_time} seconds")

# check the first 5 keys of the dict
print(list(delta_udf_marginal_dict.keys())[:5])

Time taken to load the CSV file: 2.446915864944458 seconds
[(8, 1, 1), (8, 1, 2), (8, 1, 3), (8, 1, 4), (8, 1, 5)]


In [14]:
import pandas as pd

station_df = pd.read_csv('data/station_data.csv', index_col=None)

# set the indices of the station data as a explicit column named 'idx'
station_df['idx'] = [i + 1 for i in station_df.index.tolist()]

station_df = station_df[['idx'] + [col for col in station_df.columns if col != 'idx']]

station_df

,idx,StationID,StationName,Latitude,Longitude,Capacity,CurrentInventory,Optimal Inventory,Min UDF,UDF(0),...,UDF(114),UDF(115),UDF(116),UDF(117),UDF(118),UDF(119),UDF(120),UDF(121),UDF(122),UDF(123)
0,1,0007398f39bd26118c9fdfd5197ccada,Chester Ave & 12 Ave,40.644367,-73.984276,21,12,20,2.63193,3.68365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0096a3255aa8247f5a60322c9173d420,W 56 St & 8 Ave,40.765959,-73.983135,50,2,35,26.01400,28.96080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,009750d7dc4dec3f0fc06547e41ba3a8,S Portland Ave & Hanson Pl,40.685396,-73.974315,27,4,5,24.04150,24.79040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00b598603e95ff6cd619487b42e1b87c,E 78 St & 2 Ave,40.772797,-73.955778,25,9,21,29.91800,31.81400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,00c998e78f71ad6b1a865fb2906d4326,87 St & Roosevelt Ave,40.748250,-73.879360,23,3,16,1.95563,2.76063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,2105,ff2f088627090e550307de251e788b80,Columbus Ave & W 72 St,40.777057,-73.978985,67,2,59,79.10710,83.83160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2105,2106,ff740e13f44126eac725dd9fafbd24de,E 138 St & Willow Ave,40.804168,-73.911545,18,12,8,2.01574,2.71688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2106,2107,ffa1de1039109f34b259df1275fda2a6,Dean St & Franklin Ave,40.677592,-73.955637,21,12,8,26.25480,26.80110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2107,2108,ffdca64c32febb7cf8462c1bee53cd54,E 149 St & Morris Ave,40.817340,-73.922962,35,22,11,16.91700,18.70050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
travel_time = pd.read_csv('data/duration_matrix.csv', header=0, index_col=0).to_numpy()

travel_time[1,3]

np.float64(6.96)

In [27]:
df = pd.read_csv("data/delta_udf_marginal_demo.csv")
max_transfer_idx = df.groupby(['from_surplus', 'to_deficit'])['transfers'].idxmax()
max_transfer_df = df.loc[max_transfer_idx].reset_index(drop=True)

max_transfer_df.set_index(['from_surplus', 'to_deficit'])['transfers'].to_dict()
max_transfer_df.set_index(['from_surplus', 'to_deficit', 'transfers'])['Delta UDF'].to_dict()

{(8, 1, 8): 0.6057899999999994,
 (8, 2, 8): 1.4451199999999993,
 (8, 3, 1): 0.074900000000003,
 (8, 4, 8): 0.7909199999999998,
 (8, 5, 8): 0.6524399999999995,
 (8, 6, 3): 0.22743,
 (8, 7, 4): 0.3091200000000071,
 (8, 9, 1): 0.0757699999999985,
 (8, 10, 6): 0.3614400000000001,
 (8, 12, 5): 0.4710699999999996,
 (8, 14, 3): 0.3343499999999988,
 (8, 15, 8): 0.60372,
 (8, 16, 2): 0.1419899999999998,
 (8, 17, 5): 0.8709599999999966,
 (8, 19, 8): 0.8337199999999996,
 (11, 1, 8): 0.7832899999999996,
 (11, 2, 8): 1.6226200000000013,
 (11, 3, 1): 0.177780000000002,
 (11, 4, 8): 0.9684199999999984,
 (11, 5, 8): 0.8299399999999997,
 (11, 6, 3): 0.3975500000000007,
 (11, 7, 4): 0.4863900000000001,
 (11, 9, 1): 0.1786499999999993,
 (11, 10, 6): 0.5406899999999989,
 (11, 12, 5): 0.6503599999999996,
 (11, 14, 3): 0.5044699999999995,
 (11, 15, 8): 0.7812200000000002,
 (11, 16, 2): 0.2925099999999991,
 (11, 17, 5): 1.0502500000000057,
 (11, 19, 8): 1.0112200000000016,
 (13, 1, 8): 0.7442700000000042,
 (